In [3]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm import tqdm

import cv2
from keras.callbacks import ModelCheckpoint,CSVLogger
from keras import layers #for building layers of neural net
from keras.models import Model
import time
from datetime import timedelta
import zipfile

import warnings
warnings.filterwarnings("ignore")

In [4]:
SEED_VALUE = 13
os.environ['PYTHONHASHSEED']=str(SEED_VALUE)
# random.seed(SEED_VALUE)
np.random.seed(SEED_VALUE)
tf.random.set_seed(SEED_VALUE)

In [5]:
df = pd.DataFrame()

In [6]:
def make_a_df():
    SOURCE = "/kaggle/input/persian-digits-captcha/dataset/"
    files = []
    LFilenames = []
    LText = []
    LWidth = []
    LHight = []
    for filename in tqdm(os.listdir(SOURCE)):
        file = SOURCE + filename
        if os.path.getsize(file) > 0 :
            files.append(file)
            LFilenames.append(filename)
            LText.append(filename[:-4])
            LWidth.append(cv2.imread(file).shape[0])
            LHight.append(cv2.imread(file).shape[1])

    df['Filepath'] = files
    df['Filename'] = LFilenames
    df['Text'] = LText
    df['Width'] = LWidth
    df['Hight'] = LHight
    return df
df = make_a_df()
sizeofdata = df.count()[0] #num. of data


100%|██████████| 30000/30000 [03:56<00:00, 126.69it/s]


In [7]:
df.head()

,Filepath,Filename,Text,Width,Hight
0,/kaggle/input/persian-digits-captcha/dataset/1...,19812.jpg,19812,64,306
1,/kaggle/input/persian-digits-captcha/dataset/1...,16916.jpg,16916,64,306
2,/kaggle/input/persian-digits-captcha/dataset/5...,52876.jpg,52876,64,306
3,/kaggle/input/persian-digits-captcha/dataset/7...,75892.jpg,75892,64,306
4,/kaggle/input/persian-digits-captcha/dataset/7...,793270.jpg,793270,64,306


In [8]:
def splite_data(df, percent = 0.80):
    train_size = int(df.shape[0]*percent)
    test_size = int(df.shape[0] - train_size)
    shuffled = df.sample(frac=1,random_state= SEED_VALUE)
    train_df = df[:train_size][:]
    test_df = df[-test_size:][:]

    return train_df,test_df

train_df, test_df = splite_data(df)
test_df

,Filepath,Filename,Text,Width,Hight
24000,/kaggle/input/persian-digits-captcha/dataset/8...,88798.jpg,88798,64,306
24001,/kaggle/input/persian-digits-captcha/dataset/1...,1392407.jpg,1392407,64,306
24002,/kaggle/input/persian-digits-captcha/dataset/8...,899184.jpg,899184,64,306
24003,/kaggle/input/persian-digits-captcha/dataset/9...,9046302.jpg,9046302,64,306
24004,/kaggle/input/persian-digits-captcha/dataset/3...,37735.jpg,37735,64,306
...,...,...,...,...,...
29995,/kaggle/input/persian-digits-captcha/dataset/3...,387541.jpg,387541,64,306
29996,/kaggle/input/persian-digits-captcha/dataset/5...,554645.jpg,554645,64,306
29997,/kaggle/input/persian-digits-captcha/dataset/2...,28546.jpg,28546,64,306
29998,/kaggle/input/persian-digits-captcha/dataset/3...,301696.jpg,301696,64,306


In [9]:
df.Text.str.len().value_counts()

7    10228
6    10110
5     9662
Name: Text, dtype: int64

In [10]:
Possibles = "0123456789 "
img_w = 306
img_h = 64
num_classes = len(Possibles)
max_length = df.Text.str.len().max()
batch_size = 64


In [11]:
def preprocess():
    X = np.zeros((sizeofdata,img_h,img_w,1))
    y = np.zeros((sizeofdata, 7 ,num_classes))
    for i, pic in enumerate(tqdm(df.Filename)):
        img  = cv2.imread("/kaggle/input/persian-digits-captcha/dataset/" + pic,cv2.IMREAD_GRAYSCALE)
        pic_target = pic[:-4]
        
        while len(pic_target) != 7:
            pic_target += " "
            
        img = img / 255.0
        img = np.reshape(img,(64,306,1)) 
        
        target = np.zeros((7,num_classes))
        
        for j, k in enumerate(pic_target):
            index = Possibles.find(k)
            target[j,index] = 1
            
        X[i] = img
        y[i,:] = target
        
    return X,y
    
X,y = preprocess()



100%|██████████| 30000/30000 [00:43<00:00, 688.95it/s]


In [12]:
def createmodel():
    img = layers.Input(shape = (64,306,1)) # Get image as an input of size 64,306,1
    conv1 = layers.Conv2D(32, (5, 5), padding='same', activation='relu')(img)
    conv2 = layers.Conv2D(32, (5, 5), padding='same', activation='relu')(conv1)
    mp1 = layers.MaxPooling2D(pool_size=(2,2))(conv2) 
    drop1 = layers.Dropout(0.25)(mp1)
    
    conv3 = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(drop1) 
    conv4 = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(conv3)
    mp2 = layers.MaxPooling2D(pool_size=(2,2),strides=(2,2))(conv4)  
    drop2 = layers.Dropout(0.25)(mp2)
 
    flat = layers.Flatten()(drop2) #convert the layer into 1-D

    outs = []
    for _ in range(7): #for 5 letters of captcha
        dens1 = layers.Dense(256, activation='relu')(flat)
        drop = layers.Dropout(0.25)(dens1) #drops 0.5 fraction of nodes
        res = layers.Dense(num_classes, activation='softmax')(drop)

        outs.append(res) #result of layers
    
    # Compile model and return it
    model = Model(img, outs) #create model
    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=["accuracy"])
    return model

In [13]:
#Create model
model = createmodel()
model.summary()

2023-01-16 08:08:31.994939: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-16 08:08:31.995866: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-16 08:08:32.147622: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-16 08:08:32.148509: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-16 08:08:32.149327: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 306, 1)] 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 64, 306, 32)  832         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 64, 306, 32)  25632       conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 32, 153, 32)  0           conv2d_1[0][0]                   
______________________________________________________________________________________________

In [14]:
splited = int(sizeofdata *0.8)


X_train, y_train = X[:splited], y[:splited,:]
X_test, y_test = X[splited:], y[splited:,:]

In [15]:
Name = "model_persian_captcha_v4"
csv_logger = CSVLogger('training_' + Name +'.log')
best_model = ModelCheckpoint(Name +'.h5', monitor = 'val_acc', verbose=1, 
                             save_best_only=True, mode='max')

In [16]:
#Applying the model
time_initial = time.time()

hist = model.fit(X_train, 
                 [y_train[:,0,:], y_train[:,1,:], y_train[:,2,:], y_train[:,3,:],
                  y_train[:,4,:],y_train[:,5,:],y_train[:,6,:]],
                 batch_size=64,
                 epochs=60,
                 validation_split=0.2,
                 callbacks=[best_model,csv_logger])

time_elapsed = time.time() - time_initial
print("time elapsed: ", str(timedelta(seconds=time_elapsed))[:-5])#batch size- 32 defines no. of samples per gradient update
#Validation split=0.2 splits the training set in 80-20% for training nd testing

2023-01-16 08:08:36.629889: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1504051200 exceeds 10% of free system memory.
2023-01-16 08:08:38.350588: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1504051200 exceeds 10% of free system memory.
2023-01-16 08:08:39.588513: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/60


2023-01-16 08:08:42.842703: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


300/300 [==============================] - 54s 138ms/step - loss: 13.4513 - dense_1_loss: 1.5199 - dense_3_loss: 2.0056 - dense_5_loss: 2.1710 - dense_7_loss: 2.2230 - dense_9_loss: 2.2554 - dense_11_loss: 2.0057 - dense_13_loss: 1.2708 - dense_1_accuracy: 0.4394 - dense_3_accuracy: 0.2573 - dense_5_accuracy: 0.1898 - dense_7_accuracy: 0.1658 - dense_9_accuracy: 0.1570 - dense_11_accuracy: 0.3566 - dense_13_accuracy: 0.6646 - val_loss: 8.2990 - val_dense_1_loss: 0.1265 - val_dense_3_loss: 0.8718 - val_dense_5_loss: 1.3990 - val_dense_7_loss: 1.6558 - val_dense_9_loss: 1.8015 - val_dense_11_loss: 1.5216 - val_dense_13_loss: 0.9228 - val_dense_1_accuracy: 0.9600 - val_dense_3_accuracy: 0.7027 - val_dense_5_accuracy: 0.5456 - val_dense_7_accuracy: 0.4227 - val_dense_9_accuracy: 0.3694 - val_dense_11_accuracy: 0.4815 - val_dense_13_accuracy: 0.6990
Epoch 2/60
300/300 [==============================] - 41s 138ms/step - loss: 6.2628 - dense_1_loss: 0.0835 - dense_3_loss: 0.4844 - dense_5_los

In [17]:
model.save( Name + '.h5')

In [18]:
from zipfile import ZipFile

with ZipFile( Name +'.zip', 'w') as zipObj:
   # Add multiple files to the zip
   zipObj.write(Name +'.h5')
   zipObj.write('training_' + Name +'.log')


In [ ]:
#Loss on training set
#Finding Loss on training set
preds = model.evaluate(X_train,
                       [y_train[:,0,:], y_train[:,1,:], y_train[:,2,:], y_train[:,3,:],
                        y_train[:,4,:],y_train[:,5,:],y_train[:,6,:]])
print ("Loss on training set= " + str(preds[0]))

750/750 [==============================] - 164s 218ms/step - loss: 0.4246 - dense_1_loss: 5.4783e-07 - dense_3_loss: 0.0019 - dense_5_loss: 0.0249 - dense_7_loss: 0.0551 - dense_9_loss: 0.1117 - dense_11_loss: 0.1571 - dense_13_loss: 0.0740 - dense_1_accuracy: 1.0000 - dense_3_accuracy: 0.9996 - dense_5_accuracy: 0.9953 - dense_7_accuracy: 0.9900 - dense_9_accuracy: 0.9835 - dense_11_accuracy: 0.9732 - dense_13_accuracy: 0.9861
Loss on training set= 0.4246401786804199


In [ ]:
#Finding loss on test set
preds = model.evaluate(X_test,
                       [y_test[:,0,:], y_test[:,1,:], y_test[:,2,:],
                        y_test[:,3,:],y_test[:,4,:],y_test[:,5,:],y_test[:,6,:]])
print ("Loss on testing set= " + str(preds[0]))

188/188 [==============================] - 42s 219ms/step - loss: 0.3236 - dense_1_loss: 2.8610e-09 - dense_3_loss: 1.7765e-04 - dense_5_loss: 0.0033 - dense_7_loss: 0.0399 - dense_9_loss: 0.0804 - dense_11_loss: 0.1328 - dense_13_loss: 0.0669 - dense_1_accuracy: 1.0000 - dense_3_accuracy: 0.9998 - dense_5_accuracy: 0.9985 - dense_7_accuracy: 0.9923 - dense_9_accuracy: 0.9863 - dense_11_accuracy: 0.9747 - dense_13_accuracy: 0.9860
Loss on testing set= 0.3235836327075958
